In [15]:
pip install tensorflow pandas scikit-learn


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

In [17]:
base_dir = r"C:\Users\yogit\OneDrive\Desktop\sds---try"  
train_csv_path = os.path.join(base_dir, 'train.csv')
train_image_dir = os.path.join(base_dir, 'train_dataset')
test_image_dir = os.path.join(base_dir, 'test_dataset')

In [18]:
train_labels = pd.read_csv(train_csv_path)

train_imgs, val_imgs = train_test_split(train_labels, test_size=0.2, random_state=42)

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_imgs, 
                                                    directory='train_dataset', 
                                                    x_col='File Name', 
                                                    y_col='Class', 
                                                    target_size=(224, 224), 
                                                    batch_size=32, 
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_dataframe(val_imgs, 
                                                directory='train_dataset', 
                                                x_col='File Name', 
                                                y_col='Class', 
                                                target_size=(224, 224), 
                                                batch_size=32, 
                                                class_mode='categorical')

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


c:\New folder\python\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 4960 invalid image filename(s) in x_col="File Name". These filename(s) will be ignored.
  warnings.warn(
c:\New folder\python\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1241 invalid image filename(s) in x_col="File Name". These filename(s) will be ignored.
  warnings.warn(


In [20]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_labels['Class'].unique()), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
train_generator = train_datagen.flow_from_dataframe(
    train_imgs,
    directory=train_image_dir,
    x_col='File Name',  
    y_col='Class',       
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_imgs,
    directory=train_image_dir,
    x_col='File Name',  
    y_col='Class',       
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

print("Classes in train_generator:", train_generator.class_indices)
print("Classes in val_generator:", val_generator.class_indices)

Found 4960 validated image filenames belonging to 8 classes.
Found 1241 validated image filenames belonging to 8 classes.
Classes in train_generator: {'bright dune': 0, 'crater': 1, 'dark dune': 2, 'impact ejecta': 3, 'other': 4, 'slope streak': 5, 'spider': 6, 'swiss cheese': 7}
Classes in val_generator: {'bright dune': 0, 'crater': 1, 'dark dune': 2, 'impact ejecta': 3, 'other': 4, 'slope streak': 5, 'spider': 6, 'swiss cheese': 7}


In [22]:
history = model.fit(train_generator, validation_data=val_generator, epochs=80)

Epoch 1/80


c:\New folder\python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


155/155 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.7147 - loss: 1.4764 - val_accuracy: 0.8848 - val_loss: 0.3592
Epoch 2/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.8550 - loss: 0.4838 - val_accuracy: 0.9033 - val_loss: 0.3043
Epoch 3/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.8561 - loss: 0.4510 - val_accuracy: 0.8856 - val_loss: 0.3768
Epoch 4/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.8720 - loss: 0.3976 - val_accuracy: 0.9049 - val_loss: 0.3115
Epoch 5/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.8878 - loss: 0.3387 - val_accuracy: 0.9146 - val_loss: 0.2378
Epoch 6/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.8775 - loss: 0.3435 - val_accuracy: 0.9106 - val_loss: 0.2195
Epoch 7/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 4988s 32s/step - accuracy: 0.8852 - loss: 0.3374 - val_accuracy: 0.9146 - val_loss: 0.2441
Epoch 8/80
155/155 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.8927 - loss: 0.3118 - val_accuracy: 0.9

In [23]:
model.save('model.h5')

In [24]:
test_image_files = [f for f in os.listdir(test_image_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

test_df = pd.DataFrame({'image_name': test_image_files})

print("Test DataFrame head:\n", test_df.head())

Test DataFrame head:
   image_name
0      1.jpg
1     10.jpg
2    100.jpg
3   1000.jpg
4   1001.jpg


In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_image_dir,
    x_col='image_name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 2000 validated image filenames.


In [26]:
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)


filenames = test_generator.filenames
results = pd.DataFrame({"Filename": filenames, "Predictions": predicted_classes})
results.to_csv("KBLD_submission.csv", index=False)


c:\New folder\python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 54s 830ms/step


In [27]:
output_path = 'KBLD_submission.csv'

results.to_csv(output_path, index=False)

if os.path.exists(output_path):
    print(f"File saved successfully at {output_path}")
else:
    print(f"Failed to save file at {output_path}")


File saved successfully at KBLD_submission.csv


In [28]:
for filename, prediction in zip(filenames[:10], predicted_classes[:10]):  
    print(f"Filename: {filename}, Prediction: {prediction}")


Filename: 1.jpg, Prediction: 5
Filename: 10.jpg, Prediction: 1
Filename: 100.jpg, Prediction: 4
Filename: 1000.jpg, Prediction: 4
Filename: 1001.jpg, Prediction: 4
Filename: 1002.jpg, Prediction: 4
Filename: 1003.jpg, Prediction: 4
Filename: 1004.jpg, Prediction: 4
Filename: 1005.jpg, Prediction: 4
Filename: 1006.jpg, Prediction: 5
